In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Read csv files
customers=pd.read_csv("/kaggle/input/skyhack-flyhigh/customers.csv")
call_records=pd.read_csv("/kaggle/input/skyhack-flyhigh/calls.csv")
reason2call=pd.read_csv("/kaggle/input/skyhack-flyhigh/reason.csv")
test=pd.read_csv("/kaggle/input/skyhack-flyhigh/test.csv")
sent=pd.read_csv("/kaggle/input/skyhack-flyhigh/sentiment_statistics.csv")

In [ ]:
# Class encoding for sentiments with negative and positive values
cleanup_nums = {"agent_tone":     {"angry": -1, "frustrated": -2, "neutral": 0, "calm": 1, "polite": 2},
                "customer_tone": {"angry": -1, "frustrated": -2, "neutral": 0, "calm": 1, "polite": 2}}
sent = sent.replace(cleanup_nums)

In [ ]:
sent.info()

In [ ]:
#found unique reasons
reason2call["primary_call_reason"]=reason2call["primary_call_reason"].str.replace(r"\s+"," ",regex=True)
r=reason2call["primary_call_reason"].unique()
print(len(r))
print(r)

In [ ]:
# Categorical Encoding using sklearn
from sklearn.preprocessing import OrdinalEncoder

ord_enc = OrdinalEncoder()
reason2call["primary_call_reason_ord"] = ord_enc.fit_transform(reason2call[["primary_call_reason"]])
reason2call.info()

In [ ]:
reason2call.head()

In [ ]:
call_records.info()

In [ ]:
#Str to datetime
call_records["call_start_datetime"]=pd.to_datetime(call_records["call_start_datetime"])
call_records["agent_assigned_datetime"]=pd.to_datetime(call_records["agent_assigned_datetime"])
call_records["call_end_datetime"]=pd.to_datetime(call_records["call_end_datetime"])

In [ ]:
call_records.info()

In [ ]:
# found waiting time and handling time
call_records["WaitTime"]=(call_records["agent_assigned_datetime"]-call_records["call_start_datetime"]).dt.total_seconds()/60
call_records["HandlingTime"]=(call_records["call_end_datetime"]-call_records["agent_assigned_datetime"]).dt.total_seconds()/60
call_records.info()

In [ ]:
call_records.head()

In [ ]:
#Merged dataset
merged_df = pd.merge(call_records, reason2call, on="call_id", how="inner")
merged_df = pd.merge(merged_df, sent, on="call_id", how="inner")
merged_df.head()

In [ ]:
merged_df.info()

In [ ]:
# Correlation heatmap
import matplotlib.pyplot as mp
import pandas as pd
import seaborn as sb

# Import file with data
data = merged_df

# Plotting correlation heatmap
dataplot = sb.heatmap(data.corr(numeric_only=True), cmap="YlGnBu", annot=True)

# Displaying heatmap
mp.show()

In [ ]:
# drop datetime and object attributes
merged_df_numeric = merged_df.select_dtypes(include=[np.number])
merged_df_numeric.info()

In [ ]:
merged_df_numeric.skew()

In [ ]:
df.plot("WaitTime","primary_call_reason_ord")
plt.show()

In [ ]:
df.plot("HandlingTime","primary_call_reason_ord")
plt.show()

In [ ]:
df.plot("HandlingTime","customer_tone")
plt.show()

In [ ]:
df.plot("HandlingTime","agent_tone")
plt.show()